In [44]:
import sagemaker
import boto3
from sagemaker import get_execution_role
import os

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
# additionally aws key are in ~/.aws/credentials
'''
[default]
aws_access_key_id=XXXXXXXXXXXXXXXXXXXX
aws_secret_access_key=yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
'''

# https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html

#try:
#    role = sagemaker.get_execution_role()
#except ValueError:
#    iam = boto3.client('iam')    
#    # The role was previously created from inside aws sagemaker notebook
#    # Also attached AmazonSageMakerFullAccess policy and detached AmazonSageMaker-ExecutionPolicy-20210530T161933    
#    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20210530T161933')['Role']['Arn']

iam = boto3.client('iam')
# The role was previously created from inside aws sagemaker notebook
# Also attached AmazonSageMakerFullAccess policy and detached AmazonSageMaker-ExecutionPolicy-20210530T161933    
role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20210530T161933')['Role']['Arn']

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/simon-easy-ocr-data'
#role = 'arn:aws:iam::294038372338:role/hunkimSagemaker' # this was hard coded in the original code

In [45]:
#Make a dummy model.tar.gz
import tarfile
import io

with tarfile.open('model.tar.gz', "w:gz") as tar:
  dummy = b"dummy"
  info = tarfile.TarInfo('dummy.bin')
  info.size = len(dummy)
  tar.addfile(info, io.BytesIO(dummy))

In [46]:
dummy_model = sagemaker_session.upload_data(path='model.tar.gz', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(dummy_model))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-611238842315/sagemaker/simon-easy-ocr-data/model.tar.gz


In [47]:
from sagemaker.pytorch import PyTorchModel

# FIXME: Need to upload Dummy model (perhaps only model.targ.gz)
print(dummy_model)
model = PyTorchModel(model_data=dummy_model,
                     role=role,
                     framework_version='1.5.0',
                     py_version = "py3",
                     entry_point='ocr_infer.py',
                     source_dir='code')

s3://sagemaker-us-east-1-611238842315/sagemaker/simon-easy-ocr-data/model.tar.gz


In [48]:
%%time
predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

---------------!CPU times: user 2.37 s, sys: 372 ms, total: 2.75 s
Wall time: 7min 38s


In [49]:
# Get the end point
endpoint = predictor.endpoint  
print(endpoint)
print("See the logs at", "https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/"+ endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


pytorch-inference-2021-06-08-23-50-05-821
See the logs at https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-2021-06-08-23-50-05-821


In [51]:
%%time
import boto3
import json

client = boto3.client('sagemaker-runtime')

img = open('code/example.png', 'rb').read()

response = client.invoke_endpoint(
    EndpointName=endpoint, 
    ContentType='application/x-image', 
    #Accept="application/json" ,
    Body=bytearray(img)
)

print(response['Body'].read()) 

b'[[[[1, 7], [151, 7], [151, 21], [1, 21]], "Tell me what you want to do_", 0.7820994141268869]]'
CPU times: user 81.7 ms, sys: 13.3 ms, total: 95 ms
Wall time: 1.22 s


In [52]:
sagemaker_session.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [53]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '00Z4T0DZS6AZRVE3',
   'HostId': 'CnK5fgz644eMeLyCkeMgnhut6ndFe9o3ungiLDboMOWMMdoW2bmwRHucoQo5K91+sqF0YM2mgqU=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'CnK5fgz644eMeLyCkeMgnhut6ndFe9o3ungiLDboMOWMMdoW2bmwRHucoQo5K91+sqF0YM2mgqU=',
    'x-amz-request-id': '00Z4T0DZS6AZRVE3',
    'date': 'Wed, 09 Jun 2021 00:08:10 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'pytorch-inference-2021-06-08-23-50-03-534/model.tar.gz'},
   {'Key': 'sagemaker/simon-easy-ocr-data/model.tar.gz'}]}]